# Basic usage of ABCD database

In [1]:
# %load_ext autoreload
# %autoreload 2

In [9]:
import sys
sys.path.append('..')

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from ase.io import iread, read
from utils.ext_xyz import XYZReader

from abcd import ABCD

First of all, we need to define the url of the database. It could be local or remote:

- direct access: url = 'mongodb://localhost:27017'
- api access: url = 'http://localhost/api'

using with statement to catch the riased exceptions. You may can ignore them  but in that case need to handle all the unexpected events. (cannot connect to db, lost connection, wrong filter, wrong url, etc. )

In [11]:
url = 'mongodb://localhost:27017'
abcd =  ABCD(url)

print(abcd)

MongoDatabase(url=localhost:27017, db=abcd, collection=atoms)


In [12]:
abcd.print_info()

================== ABCD MongoDB ==================
      type: mongodb
      host: localhost
      port: 27017
        db: abcd
collection: atoms
number of confs: 5195


## Cleanup 

WARNING!! Remove all elements from the database.
Only supported in the case of local access

In [13]:
with abcd as db:
    db.destroy()

## Uploading configurations

In [23]:


direcotry = Path('data/')
file = direcotry / 'bcc_bulk_54_expanded_2_high.xyz'
# file = direcotry / 'GAP_1.xyz'

Uploading configurations on-by-one directly from an ase atoms object:

In [9]:
%%time
with abcd as db:

    for atoms in iread(file.as_posix(), index=slice(None)):
        
        # Hack to fix the representation of forces
        
        atoms.calc.results['forces'] = atoms.arrays['force']
        del(atoms.arrays['force'])
            
        db.push(atoms)
    

CPU times: user 1.73 s, sys: 80.7 ms, total: 1.81 s
Wall time: 2.14 s


Reading the trajectory from file:

In [10]:
%%time
traj = read(file.as_posix(), index=slice(None))
len(traj)

CPU times: user 1.03 s, sys: 36.6 ms, total: 1.07 s
Wall time: 1.14 s


In [11]:
%%time
with XYZReader(file) as reader:
    traj = list(reader.read_atoms(forces_label='force'))


CPU times: user 471 ms, sys: 21.1 ms, total: 492 ms
Wall time: 510 ms


Pushing the whole trajectory to the database:

In [12]:
%%time
db.push(traj)

CPU times: user 193 ms, sys: 7.76 ms, total: 200 ms
Wall time: 225 ms


Uploading a whole file and injecting to the database on the server side:

In [13]:
%%time
with abcd as db:
    db.upload(file.as_posix())

CPU times: user 1.41 s, sys: 44.1 ms, total: 1.45 s
Wall time: 1.56 s


An alternative way to upload file to database:

In [14]:
%%time
with abcd as db, XYZReader(file) as reader:
    db.push(reader.read_atoms(forces_label='force'))

CPU times: user 508 ms, sys: 7.69 ms, total: 515 ms
Wall time: 531 ms


In [15]:
abcd.info()

{'host': 'localhost',
 'port': 27017,
 'db': 'abcd',
 'collection': 'atoms',
 'number of confs': 8000}

## Uploading


In [14]:
for file in Path('data/').glob('*.xyz'):
    print(file)
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms())
        
    traj = read(file.as_posix(), index=slice(None))
    db.push(traj)

data/bcc_quadvacancy_124_high.xyz
data/bcc_bulk_54_high.xyz
data/bcc_monovacancy_53_high.xyz
data/bcc_bulk_54_expanded_high.xyz
data/bcc_doublevacancy_126_2NN_high.xyz
data/bcc_primitive_high.xyz
data/bcc_bulk_128_expanded_high.xyz
data/GAP_3.xyz
data/Fe_bcc_bulk_vac_multivac_surf_gamma_int_diint.xyz
data/GAP_2.xyz
data/bcc_trivacancy_111_125_high.xyz
data/GAP_1.xyz
data/GAP_5.xyz
data/GAP_4.xyz
data/bcc_primitive_contracted_high.xyz
data/GAP_6.xyz
data/bcc_self_di_interstitial_npc_130_high.xyz
data/bcc_trivacancy_110_125_high.xyz
data/bcc_bulk_54_expanded_2_high.xyz
data/bcc_trivacancy_100_125_high.xyz
data/bcc_quinvacancy_123_high.xyz
data/bcc_bulk_128_high.xyz
data/bcc_primitive_expanded_high.xyz
data/bcc_monovacancy_127_high.xyz
data/bcc_surface.xyz
data/bcc_gamma_surface.xyz
data/bcc_doublevacancy_126_high.xyz
data/bcc_self_interstitial_high.xyz
data/bcc_doublevacancy_126_1NN_high.xyz


In [15]:
for file in Path('GB_alphaFe_001/tilt/').glob('*.xyz'):
    print(file)
    gb_params = {
        'name': 'alphaFe',
        'type': 'tilt',
        'params': file.name[:-4]
        
    }    

    traj = read(file.as_posix(), index=slice(None))
    db.push(traj, extra_info={'GB_params': gb_params})

GB_alphaFe_001/tilt/00183979100_v6bxv2_tv0.3bxv0.3_d1.4z_traj.xyz
GB_alphaFe_001/tilt/0014111380_v6bxv2_tv0.1bxv0.1_d1.6z_traj.xyz
GB_alphaFe_001/tilt/0014640370_v6bxv2_tv0.2bxv0.4_d1.8z_traj.xyz
GB_alphaFe_001/tilt/001860514150_v6bxv2_tv0.2bxv0.3_d1.4z_traj.xyz
GB_alphaFe_001/tilt/0016738230_v6bxv2_tv0.1bxv0.0_d1.6z_traj.xyz
GB_alphaFe_001/tilt/0012807140_v6bxv2_tv0.4bxv0.0_d1.8z_traj.xyz
GB_alphaFe_001/tilt/00195301120_v6bxv2_tv0.4bxv0.3_d1.9z_traj.xyz
GB_alphaFe_001/tilt/00150037150_v6bxv2_tv0.0bxv0.2_d1.5z_traj.xyz
GB_alphaFe_001/tilt/0015811590_v6bxv2_tv0.4bxv0.4_d1.6z_traj.xyz
GB_alphaFe_001/tilt/0017374340_v6bxv2_tv0.4bxv0.0_d1.7z_traj.xyz
GB_alphaFe_001/tilt/00148895110_v6bxv2_tv0.4bxv0.2_d1.6z_traj.xyz
GB_alphaFe_001/tilt/00125993130_v6bxv2_tv0.2bxv0.2_d1.7z_traj.xyz
GB_alphaFe_001/tilt/0017732450_v6bxv2_tv0.2bxv0.1_d1.9z_traj.xyz
GB_alphaFe_001/tilt/0016193350_v6bxv2_tv0.4bxv0.4_d2.0z_traj.xyz
GB_alphaFe_001/tilt/00147267160_v6bxv2_tv0.4bxv0.4_d1.6z_traj.xyz
GB_alphaFe_001/ti

In [ ]:
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms(), extra_info={'GB_params': gb_params})
